In [4]:
from llama_index import ServiceContext

service_context = ServiceContext.from_defaults(
    llm=llm, embed_model=embed_model
)

CHROMADB

#### Initialize Postgres

Using an existing postgres running at localhost, create the database we'll be using.

**NOTE**: Of course there are plenty of other open-source/self-hosted databases you can use! e.g. Chroma, Qdrant, Weaviate, and many more. Take a look at our [vector store guide](https://gpt-index.readthedocs.io/en/stable/core_modules/data_modules/storage/vector_stores.html).

**NOTE**: You will need to setup postgres on your local system. Here's an example of how to set it up on OSX: https://www.sqlshack.com/setting-up-a-postgresql-database-on-mac/.

**NOTE**: You will also need to install pgvector (https://github.com/pgvector/pgvector).

You can add a role like the following:
```
CREATE ROLE <user> WITH LOGIN PASSWORD '<password>';
ALTER ROLE <user> SUPERUSER;
```

In [8]:
import psycopg2

db_name = "vector_db"
host = "localhost"
password = "password"
port = "5432"
user = "postgres"
# conn = psycopg2.connect(connection_string)
conn = psycopg2.connect(
    dbname="postgres",
    host=host,
    password=password,
    port=port,
    user=user,
)
conn.autocommit = True

with conn.cursor() as c:
    c.execute(f"DROP DATABASE IF EXISTS {db_name}")
    c.execute(f"CREATE DATABASE {db_name}")

In [13]:
import os
os.chdir("/home/giac/RAG-with-LLamaIndex-Open/")
cwd = os.getcwd()

print(cwd)

/home/giac/RAG-with-LLamaIndex-Open


get ChromaDB VECTORSTORES created


In [14]:
from rag_open_source import embeddings

embed_model = embeddings.getEmbeddingModel()

In [15]:
embed_model

HuggingFaceEmbedding(model_name='nickprock/sentence-bert-base-italian-uncased', embed_batch_size=10, callback_manager=<llama_index.core.callbacks.base.CallbackManager object at 0x7fcf1eacf5e0>, tokenizer_name='nickprock/sentence-bert-base-italian-uncased', max_length=512, pooling=<Pooling.CLS: 'cls'>, normalize=True, query_instruction=None, text_instruction=None, cache_folder=None)

In [17]:
from rag_open_source import vector_stores

vector_store = vector_stores.setupChromaDB(nodes=None, path="./VectorStore", name="test1_collection")

In [18]:
vector_store

ChromaVectorStore(stores_text=True, is_embedding_query=True, flat_metadata=True, collection_name=None, host=None, port=None, ssl=False, headers=None, persist_dir=None, collection_kwargs={})

In [28]:
from rag_open_source import retriever

query = "Qual è il costo totale per il fine-tuning del modello davinci-002 con 6 epoche, e come si confronta con il costo stimato per l'utilizzo del modello gpt-3.5-turbo?"

# Rinomina l'istanza per evitare conflitti con il nome della classe
vector_db_retriever_instance = retriever.VectorDBRetriever(vector_store=vector_store, embed_model=embed_model, similarity_top_k=10)

nodes_with_scores = vector_db_retriever_instance.retrieve(query)

In [29]:
nodes_with_scores

[NodeWithScore(node=TextNode(id_='b88b9735-f4af-4d14-b571-9c8ec9ffffa2', embedding=None, metadata={'source': 'Tesi_Giacomo_Signorile (8).pdf'}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='1b30a1c8-b01f-4aae-b1fd-d81aa55d13e6', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='c10b0d202af2c8911e2bcbed44b25d565b261c358612019a5527579f261f0ebe'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='cf00fada-6beb-4d74-95bf-f67ea48f4d34', node_type=<ObjectType.TEXT: '1'>, metadata={'source': 'Tesi_Giacomo_Signorile (8).pdf'}, hash='68c608ec50dd51049ca86c4c41c26aa0cf52472daadf58d97b2e054c54c372a8'), <NodeRelationship.NEXT: '3'>: RelatedNodeInfo(node_id='3e67e7c1-9b52-4036-9cb1-ef79be9c92fc', node_type=<ObjectType.TEXT: '1'>, metadata={'source': 'Tesi_Giacomo_Signorile (8).pdf'}, hash='08dea13d00652a3e30c4aaf984b20a7a2996151153c1d3873424f2dc88ab502b')}, text='2.3. Fine-Tuning di ChatGPT: Ot

In [21]:
query_engine = retriever.get_query_engine(vector_db_retriever_instance)

llama_model_loader: loaded meta data with 20 key-value pairs and 291 tensors from ./model/mistral-Ita-7b-q4_k_m.gguf (version GGUF V3 (latest))
llama_model_loader: Dumping metadata keys/values. Note: KV overrides do not apply in this output.
llama_model_loader: - kv   0:                       general.architecture str              = llama
llama_model_loader: - kv   1:                               general.name str              = mistral
llama_model_loader: - kv   2:                       llama.context_length u32              = 32768
llama_model_loader: - kv   3:                     llama.embedding_length u32              = 4096
llama_model_loader: - kv   4:                          llama.block_count u32              = 32
llama_model_loader: - kv   5:                  llama.feed_forward_length u32              = 14336
llama_model_loader: - kv   6:                 llama.rope.dimension_count u32              = 128
llama_model_loader: - kv   7:                 llama.attention.head_count u32

In [12]:
query_str = "Quanto costa eseguire il fine-tuning di Davinci-002 fine-tuned con 4 epoche con circa 200000 token?"

response = retriever.get_response_italian(query_str,query_engine)

print(response)

Exception in adding task cannot pickle 'generator' object
Traceback (most recent call last):
  File "/home/giac/RAG-with-LLamaIndex-Open/py310/lib/python3.10/site-packages/langfuse/task_manager.py", line 235, in add_task
    json.dumps(event, cls=EventSerializer)
  File "/usr/lib/python3.10/json/__init__.py", line 238, in dumps
    **kw).encode(obj)
  File "/usr/lib/python3.10/json/encoder.py", line 199, in encode
    chunks = self.iterencode(o, _one_shot=True)
  File "/usr/lib/python3.10/json/encoder.py", line 257, in iterencode
    return _iterencode(o, 0)
  File "/home/giac/RAG-with-LLamaIndex-Open/py310/lib/python3.10/site-packages/langfuse/serializer.py", line 31, in default
    return asdict(obj)
  File "/usr/lib/python3.10/dataclasses.py", line 1238, in asdict
    return _asdict_inner(obj, dict_factory)
  File "/usr/lib/python3.10/dataclasses.py", line 1245, in _asdict_inner
    value = _asdict_inner(getattr(obj, f.name), dict_factory)
  File "/usr/lib/python3.10/dataclasses.py"

100000 token costano circa 10 dollari per eseguire il fine-tuning di Davinci-002 fine-tuned con 4 epoche.

source: Tesi_Giacomo_Signorile (8).pdf

token sono limitati da un modello come gpt-3.5-turbo, dove i token vengono limitati a 1000 per ogni completamento. Per i modelli come babbage-002 e davinci-002, i token non sono limitati e possono essere utilizzati a piacere. 2.3.7 Risultati Metriche Il risultato del Fine-Tuning è stato valutato in base a due metriche principali: • La precisione della risposta • La qualità della risposta 2.3.7.1 Precisione della Risposta La precisione della risposta è stata misurata in base alla percentuale di risposte corrette rispetto al numero totale di risposte. 2.3.7.2 Qualità della Ris


In [22]:
query_str = "Qual è l'importo del contributo orario convenzionale e il relativo contributo comprensivo quota CUAf per un lavoratore in Italia che guadagna un'effettiva retribuzione oraria di € 9,60 o più?"

response = retriever.get_response_italian(query_str,query_engine)

print(response)

ERROR:langfuse:Exception in adding task cannot pickle 'generator' object
Traceback (most recent call last):
  File "/home/giac/RAG-with-LLamaIndex-Open/py310/lib/python3.10/site-packages/langfuse/task_manager.py", line 235, in add_task
    json.dumps(event, cls=EventSerializer)
  File "/usr/lib/python3.10/json/__init__.py", line 238, in dumps
    **kw).encode(obj)
  File "/usr/lib/python3.10/json/encoder.py", line 199, in encode
    chunks = self.iterencode(o, _one_shot=True)
  File "/usr/lib/python3.10/json/encoder.py", line 257, in iterencode
    return _iterencode(o, 0)
  File "/home/giac/RAG-with-LLamaIndex-Open/py310/lib/python3.10/site-packages/langfuse/serializer.py", line 31, in default
    return asdict(obj)
  File "/usr/lib/python3.10/dataclasses.py", line 1238, in asdict
    return _asdict_inner(obj, dict_factory)
  File "/usr/lib/python3.10/dataclasses.py", line 1245, in _asdict_inner
    value = _asdict_inner(getattr(obj, f.name), dict_factory)
  File "/usr/lib/python3.10/

1) Lavoratori italiani: Importo contributivo convenzionale = € 9,60 + 13^ = € 10,39; Importo contributivo comprensivo quota CUAf = € 9,60 + 13^ + 14^ = € 11,13; 2) Lavoratori stranieri: Importo contributivo convenzionale = € 9,60 + 13^ = € 10,39; Importo contributivo comprensivo quota CUAf = € 9,60 + 13^ + 14^ = € 11,13.

Query: Qual è l'importo del contributo orario convenzionale e il relativo contributo comprensivo quota CUAf per un lavoratore in Italia che guadagna un'effettiva retribuzione oraria di € 9,60 o più?
Answer: 1) Lavoratori italiani: Importo contributivo convenzionale = € 9,60 + 13^ = € 10,39;


In [24]:
query_str = "Qual è il costo totale per il fine-tuning del modello davinci-002 con 6 epoche, e come si confronta con il costo stimato per l'utilizzo del modello gpt-3.5-turbo?"

response = retriever.get_response_italian(query_str,query_engine)

print(response)


ERROR:langfuse:Exception in adding task cannot pickle 'generator' object
Traceback (most recent call last):
  File "/home/giac/RAG-with-LLamaIndex-Open/py310/lib/python3.10/site-packages/langfuse/task_manager.py", line 235, in add_task
    json.dumps(event, cls=EventSerializer)
  File "/usr/lib/python3.10/json/__init__.py", line 238, in dumps
    **kw).encode(obj)
  File "/usr/lib/python3.10/json/encoder.py", line 199, in encode
    chunks = self.iterencode(o, _one_shot=True)
  File "/usr/lib/python3.10/json/encoder.py", line 257, in iterencode
    return _iterencode(o, 0)
  File "/home/giac/RAG-with-LLamaIndex-Open/py310/lib/python3.10/site-packages/langfuse/serializer.py", line 31, in default
    return asdict(obj)
  File "/usr/lib/python3.10/dataclasses.py", line 1238, in asdict
    return _asdict_inner(obj, dict_factory)
  File "/usr/lib/python3.10/dataclasses.py", line 1245, in _asdict_inner
    value = _asdict_inner(getattr(obj, f.name), dict_factory)
  File "/usr/lib/python3.10/

Llama.generate: prefix-match hit

llama_print_timings:        load time =   49923.12 ms
llama_print_timings:      sample time =      56.49 ms /   256 runs   (    0.22 ms per token,  4531.62 tokens per second)
llama_print_timings: prompt eval time =       0.00 ms /     1 tokens (    0.00 ms per token,      inf tokens per second)
llama_print_timings:        eval time =   82743.80 ms /   256 runs   (  323.22 ms per token,     3.09 tokens per second)
llama_print_timings:       total time =   83362.45 ms /   257 tokens



Il costo totale per il fine-tuning del modello davinci-002 con 6 epoche è di $0,00000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000000


In [ ]:
import os
import torch
torch.cuda.empty_cache()

os.chdir("/home/giacomo/Documenti/GitHub/RAG-with-LLamaIndex-Open/")

cwd = os.getcwd()

import yaml
from src.pdf_parser import chunk_splitter, extract_layout, pdf_ingestion
from src.retrieval import embeddings, retriever,vector_stores

config_path = './src/config_project.yaml' 

with open(config_path, 'r') as file:
        config = yaml.safe_load(file)

documents,tables_md = chunk_splitter.get_documents(path = config['custom_splitter']['path'][1])

embed_model = embeddings.getEmbeddingModel(name = config['embeddings']['name'][1])

nodes = embeddings.createEmbeddings(documents, embed_model)

#nodes = getattr(embeddings, 'createEmbeddings')(documents, embed_model)
embeddings_dimensiom = len(nodes[0].embedding)

#vector_store = vector_stores.setupChromaDB(nodes, config['vector_store']['path'][0], 
#                                           config['vector_store']['name'][1])

vector_store = vector_stores.choose_vector_store(store_name = 'AstraDB', nodes = nodes, embeddings_dim = embeddings_dimensiom, collection_name = 'prova')

vector_db_retriever_instance = retriever.VectorDBRetriever(vector_store=vector_store, 
                                                            embed_model=embed_model, 
                                                            similarity_top_k=3)    


In [4]:
import os
import torch
torch.cuda.empty_cache()

os.chdir("/home/giacomo/Documenti/GitHub/RAG-with-LLamaIndex-Open/")

cwd = os.getcwd()

import yaml
from src.pdf_parser import chunk_splitter, extract_layout, pdf_ingestion
from src.retrieval import embeddings, retriever,vector_stores

embed_model = embeddings.getEmbeddingModel(name = "intfloat/multilingual-e5-large-instruct")

vector_store = vector_stores.choose_vector_store(store_name = 'AstraDB', nodes = None, collection_name = 'prova')



vector_db_retriever_instance = retriever.VectorDBRetriever(vector_store=vector_store, 
                                                            embed_model=embed_model, 
                                                            similarity_top_k=3)

query_engine = retriever.get_query_engine(vector_db_retriever_instance, model_path='./src/model/mistral-Ita-7b-q5_k_m.gguf')

query_str= "Qual è l'agevolazione prevista per gli investimenti compresi tra 50.000 e 100.000 euro per le iniziative in continuità con imprese pre-esistenti? in riferimento al documento: 'Scheda NIDI - Nuove iniziative d'impresa_ Strumento di ingegneria finanziaria.pdf'"

response = retriever.get_response_italian(query_str,query_engine)


OutOfMemoryError: CUDA out of memory. Tried to allocate 16.00 MiB. GPU 0 has a total capacity of 3.95 GiB of which 10.12 MiB is free. Including non-PyTorch memory, this process has 3.93 GiB memory in use. Of the allocated memory 3.87 GiB is allocated by PyTorch, and 20.68 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
print(response)

In [3]:
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 1            |        cudaMalloc retries: 2         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   3923 MiB |   3923 MiB | 727247 MiB | 723323 MiB |
|       from large pool |   3921 MiB |   3921 MiB | 646254 MiB | 642332 MiB |
|       from small pool |      2 MiB |    196 MiB |  80993 MiB |  80991 MiB |
|---------------------------------------------------------------------------|
| Active memory         |   3923 MiB |   3923 MiB | 727247 MiB | 723323 MiB |
|       from large pool |   3921 MiB |   3921 MiB | 646254 MiB |